In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from time import sleep
from parsel import Selector
import json
import glob
import os
import requests
import urllib.request 


In [2]:
def get_URL():
    i = 1
    urls = []        
    lsURL = driver.find_elements_by_class_name('icon_commend')
    for url in lsURL:
        if url.get_attribute("href"):
            str_url = str(url.get_attribute("href"))
            urls.append(str_url[:-12])
            print(str(i) + "\t" + str_url[:-12])
            i +=1
    
    print( "A total of {total} links".format( total = len(urls) ))

    return urls

In [3]:
def get_info(lsUrls):
    count = 0
    comment_url = 'https://usi-saas.vnexpress.net/index/get?callback=okmen&offset=0&limit=1000&frommobile=0&sort=like&objectid='
    header_keys = ['No','Title', 'URL', 'ObjectID', 'SiteID', 'ObjectType', "JSON_URL"]
    df = pd.DataFrame(columns = header_keys)
    error = 0
    flag = False
    for url in lsUrls:
        try:
            driver.get(str(url))
            sleep(0.5)
        except NameError:
            print("\n==========> Error: " + str(url))
            error +=1
            continue
        
        print("Loading page: " + str(url) )
        
        sel = Selector(text=driver.page_source)
        title = sel.xpath('//*[starts-with(@class, "title_news_detail")]/text()').extract_first()
        
        if driver.find_elements_by_xpath('//*[@id="social_like"]/div[1]'):
            parent_tag = driver.find_elements_by_xpath('//*[@id="social_like"]/div[1]')
            print("===== XPath selector")
        else:
            try:
                parent_tag = driver.find_element_by_css_selector('div.item_social.font_icon')
                flag = True
                print("===== CSS selector")
            except:
                print("\n==========> Error: " + str(url))
                print("Not found elements\n\n")
                error +=1
                continue
            
        #The way to get attribute for Xpath
        if(flag == True):
            objid = parent_tag.get_attribute('data-component-objectid')
            siteid = parent_tag.get_attribute('data-component-siteid')
            objtype = parent_tag.get_attribute('data-objecttype')
            flag = False
        else:
            for obj in parent_tag:
                objid = obj.get_attribute('data-component-objectid')
                siteid = obj.get_attribute('data-component-siteid')
                objtype = obj.get_attribute('data-objecttype')
                break
            
        direct_comment_url = comment_url + str(objid) + "&siteid=" + str(siteid) + "&objecttype=" + str(objtype)
        curURL = driver.current_url

        print(str(count) + " -- Get information:" + " " + str(curURL))
        
        if title: title = title.strip()
            
        try:
            df = df.append(pd.Series([count ,title, curURL, objid, siteid, objtype, direct_comment_url], index= df.columns ), ignore_index=True)
            print("Appending " + str(curURL) + " into dataframe ==============> DONE")
        except NameError:
            print("\n==========> Error: " + str(curURL))
            error +=1
            continue
        
        print()
        count +=1

    print("\n Collected " + str(count)+"/"+str(count+error)+ " links \n Error links: " + str(error)+"/"+str(error + count))
    print(" Error rate: {}".format( count/(count+error) ))
    if not os.path.isfile('urls.csv'):
        print("\n=========== Create 'data.csv' ===========\n")
        df.to_csv('urls.csv', index = False)
        print("Writeout CSV file ==============> DONE\n")
    else:
        df.to_csv('urls.csv', mode='a', header = None, index = False)
        print("Writeout CSV file ==============> DONE\n")  
        
    return df


In [4]:
def write_to_json(data):
    i = 1
    strRep = "/**/ typeof okmen === 'function' && okmen({\"error\":0,\"errorDescription\":\"\",\"iscomment\":1,"
    folder = "json/"
    for url in data["JSON_URL"]:
        sleep(2)
        response = urllib.request.urlopen(url)
        str_response = response.read().decode('utf-8')
        pre = str_response.replace(strRep, "{", 1)
        content = json.loads(pre[:-2])

        with open("json/%i.json" %i, 'w') as ofile:
            json.dump(content, ofile, sort_keys = True, indent = 4, ensure_ascii = False)
            print("Writeout {idx}.json -----> DONE".format(idx = i))
            
        i +=1

In [5]:
def parse_json(data_json):
    objects = data_json["data"]["items"]
    content = []
    article_id =[]
    comment_id = []
    userid = []
    userlike = []
    idx = 0
    for obj in objects:
        article_id.append(obj["article_id"])
        content.append(obj["content"])
        comment_id.append(obj["comment_id"])
        userlike.append(obj["userlike"])
        userid.append(obj["userid"])
        
        if obj["replys"]:
            rep_objects = obj["replys"]["items"]
            for rep in rep_objects:
                article_id.append(rep["article_id"])
                content.append(rep["content"])
                comment_id.append(rep["comment_id"])
                userlike.append(rep["userlike"])
                userid.append(rep["userid"])
    
    return (article_id, userid, comment_id, content, userlike)
                

In [6]:
def save_to_csv():
    list_json_file = glob.glob('json/*.json')
    list_json_file.sort()
    i = 1
    for filename in list_json_file:
        print("Opening " + str(filename))
        with open(filename) as json_file:
            data_json = json.load(json_file)
            
            #parse json to convert to text
            articleID, userID, commentID, content_comment, userLike = parse_json(data_json)

            # create column name to store in dataframe
            d = {
                'article_id':articleID,
                'user_id' : userID,
                'comment_id': commentID,
                'content': content_comment,
                'userlike': userLike
            }

            df_from_json_file = pd.DataFrame(d)
            df_from_json_file.to_csv("data/%i.csv"%i)
            print("======== Saving {idx}.csv ========".format(idx = i))
            i +=1


# Executing

In [7]:
#edit path if using windows
driver = webdriver.Chrome('chromedriver_linux64/chromedriver')
driver.get("https://vnexpress.net")
sleep(0.5)

In [8]:
lsUrls = get_URL()

1	https://vnexpress.net/thoi-su/linh-cuu-nguyen-chu-tich-nuoc-le-duc-anh-duoc-dua-vao-tp-hcm-3917744.html
2	https://vnexpress.net/phap-luat/loan-da-trong-quan-karaoke-khach-ban-hai-nhan-vien-phuc-vu-3917894.html
3	https://vnexpress.net/phap-luat/100-canh-sat-da-nang-dot-kich-quan-bar-tai-da-nang-3917919.html
4	https://vnexpress.net/giai-tri/bao-vat-quoc-gia-vie-t-nam-ho-ng-30-do-ve-sinh-khong-du-ng-ca-ch-3917966.html
5	https://vnexpress.net/the-gioi/hanh-trinh-tu-tiep-vien-hang-khong-toi-hoang-hau-cua-nu-tuong-thai-lan-3917769.html
6	https://vnexpress.net/phap-luat/dam-chet-hai-ban-nhau-trong-hon-chien-o-quan-bia-3917875.html
7	https://vnexpress.net/du-lich/ly-do-viet-nam-co-nhieu-canh-dep-nhung-khong-hut-khach-bang-singapore-3917826.html
8	https://vnexpress.net/khoa-hoc/32-doi-thi-sap-tranh-tai-tai-chung-ket-cuoc-thi-robocon-2019-3917847.html
9	https://vnexpress.net/giai-tri/park-yoo-chun-tung-bi-ban-gai-cu-uy-hiep-bang-anh-chup-len-3917986.html
10	https://vnexpress.net/oto-xe-may/por

88	https://vnexpress.net/the-gioi/cuoc-dao-chinh-chet-yeu-cua-phe-doi-lap-venezuela-3917239.html
89	https://vnexpress.net/bong-da/messi-thua-nhan-an-may-voi-cu-sut-phat-vao-luoi-liverpool-3917202.html
90	https://vnexpress.net/the-gioi/ly-do-hoang-hau-thai-lan-quy-rap-truoc-chong-trong-le-sac-phong-3917222.html
91	https://vnexpress.net/bong-da/van-dijk-bi-nghi-cau-cuu-dong-doi-khi-doi-dau-messi-3917470.html
92	https://vnexpress.net/giai-tri/gia-dinh-dong-nghiep-thuong-tiec-le-binh-trong-tang-le-3917137.html
93	https://ngoisao.net/hau-truong/ve-nam-tinh-cua-nu-dien-vien-ve-nha-di-con-3917682.html
94	https://ione.vnexpress.net/tin-tuc/sao/us-uk/jaden-smith-con-dau-bong-dang-cua-the-karate-kid-3917737.html
A total of 94 links


In [9]:
data = get_info(lsUrls)

Loading page: https://vnexpress.net/thoi-su/linh-cuu-nguyen-chu-tich-nuoc-le-duc-anh-duoc-dua-vao-tp-hcm-3917744.html
===== XPath selector
0 -- Get information: https://vnexpress.net/thoi-su/linh-cuu-nguyen-chu-tich-nuoc-le-duc-anh-duoc-dua-vao-tp-hcm-3917744.html
Appending https://vnexpress.net/thoi-su/linh-cuu-nguyen-chu-tich-nuoc-le-duc-anh-duoc-dua-vao-tp-hcm-3917744.html into dataframe ==============> DONE

Loading page: https://vnexpress.net/phap-luat/loan-da-trong-quan-karaoke-khach-ban-hai-nhan-vien-phuc-vu-3917894.html
===== XPath selector
1 -- Get information: https://vnexpress.net/phap-luat/loan-da-trong-quan-karaoke-khach-ban-hai-nhan-vien-phuc-vu-3917894.html
Appending https://vnexpress.net/phap-luat/loan-da-trong-quan-karaoke-khach-ban-hai-nhan-vien-phuc-vu-3917894.html into dataframe ==============> DONE

Loading page: https://vnexpress.net/phap-luat/100-canh-sat-da-nang-dot-kich-quan-bar-tai-da-nang-3917919.html
===== XPath selector
2 -- Get information: https://vnexpre

Loading page: https://vnexpress.net/giao-duc/nhung-truong-ha-noi-co-hoc-phi-lop-1-hang-tram-trieu-dong-3914282.html

==========> Error: https://vnexpress.net/giao-duc/nhung-truong-ha-noi-co-hoc-phi-lop-1-hang-tram-trieu-dong-3914282.html
Not found elements


Loading page: https://vnexpress.net/kinh-doanh/vietjet-de-xuat-xay-san-bay-vingroup-muon-hut-nguoi-tai-ve-nuoc-3917729.html
===== XPath selector
17 -- Get information: https://vnexpress.net/kinh-doanh/vietjet-de-xuat-xay-san-bay-vingroup-muon-hut-nguoi-tai-ve-nuoc-3917729.html
Appending https://vnexpress.net/kinh-doanh/vietjet-de-xuat-xay-san-bay-vingroup-muon-hut-nguoi-tai-ve-nuoc-3917729.html into dataframe ==============> DONE

Loading page: https://video.vnexpress.net/tin-tuc/xa-hoi/bien-hoa-ngap-sau-mua-nguoi-dan-dat-xe-ve-nha-trong-dem-3917734.html

==========> Error: https://video.vnexpress.net/tin-tuc/xa-hoi/bien-hoa-ngap-sau-mua-nguoi-dan-dat-xe-ve-nha-trong-dem-3917734.html
Not found elements


Loading page: https://vnexp

Loading page: https://vnexpress.net/infographics/600-ban-cua-messi-cho-barca-den-nhu-the-nao-3917617.html
===== XPath selector
36 -- Get information: https://vnexpress.net/infographics/600-ban-cua-messi-cho-barca-den-nhu-the-nao-3917617.html
Appending https://vnexpress.net/infographics/600-ban-cua-messi-cho-barca-den-nhu-the-nao-3917617.html into dataframe ==============> DONE

Loading page: https://vnexpress.net/infographics/bo-ba-smartphone-tam-gia-7-trieu-dong-do-suc-3916633.html
===== XPath selector
37 -- Get information: https://vnexpress.net/infographics/bo-ba-smartphone-tam-gia-7-trieu-dong-do-suc-3916633.html
Appending https://vnexpress.net/infographics/bo-ba-smartphone-tam-gia-7-trieu-dong-do-suc-3916633.html into dataframe ==============> DONE

Loading page: https://e.vnexpress.net/news/news/vietnam-holds-state-funeral-for-former-president-le-duc-anh-3917773.html

==========> Error: https://e.vnexpress.net/news/news/vietnam-holds-state-funeral-for-former-president-le-duc-anh-

Loading page: https://vnexpress.net/doi-song/nha-ong-sai-gon-thua-nang-thua-gio-3917630.html
===== XPath selector
55 -- Get information: https://vnexpress.net/doi-song/nha-ong-sai-gon-thua-nang-thua-gio-3917630.html
Appending https://vnexpress.net/doi-song/nha-ong-sai-gon-thua-nang-thua-gio-3917630.html into dataframe ==============> DONE

Loading page: https://vnexpress.net/doi-song/ty-phu-warren-buffett-mang-bao-nhieu-tien-mat-trong-vi-3917668.html
===== XPath selector
56 -- Get information: https://vnexpress.net/doi-song/ty-phu-warren-buffett-mang-bao-nhieu-tien-mat-trong-vi-3917668.html
Appending https://vnexpress.net/doi-song/ty-phu-warren-buffett-mang-bao-nhieu-tien-mat-trong-vi-3917668.html into dataframe ==============> DONE

Loading page: https://vnexpress.net/so-hoa/5-smartphone-khong-lo-gia-re-3917879.html
===== XPath selector
57 -- Get information: https://vnexpress.net/so-hoa/5-smartphone-khong-lo-gia-re-3917879.html
Appending https://vnexpress.net/so-hoa/5-smartphone-khon

Loading page: https://vnexpress.net/the-gioi/cuoc-dao-chinh-chet-yeu-cua-phe-doi-lap-venezuela-3917239.html
===== XPath selector
76 -- Get information: https://vnexpress.net/the-gioi/cuoc-dao-chinh-chet-yeu-cua-phe-doi-lap-venezuela-3917239.html
Appending https://vnexpress.net/the-gioi/cuoc-dao-chinh-chet-yeu-cua-phe-doi-lap-venezuela-3917239.html into dataframe ==============> DONE

Loading page: https://vnexpress.net/bong-da/messi-thua-nhan-an-may-voi-cu-sut-phat-vao-luoi-liverpool-3917202.html
===== XPath selector
77 -- Get information: https://vnexpress.net/bong-da/messi-thua-nhan-an-may-voi-cu-sut-phat-vao-luoi-liverpool-3917202.html
Appending https://vnexpress.net/bong-da/messi-thua-nhan-an-may-voi-cu-sut-phat-vao-luoi-liverpool-3917202.html into dataframe ==============> DONE

Loading page: https://vnexpress.net/the-gioi/ly-do-hoang-hau-thai-lan-quy-rap-truoc-chong-trong-le-sac-phong-3917222.html
===== XPath selector
78 -- Get information: https://vnexpress.net/the-gioi/ly-do-ho

In [10]:
write_to_json(data)

Writeout 1.json -----> DONE
Writeout 2.json -----> DONE
Writeout 3.json -----> DONE
Writeout 4.json -----> DONE
Writeout 5.json -----> DONE
Writeout 6.json -----> DONE
Writeout 7.json -----> DONE
Writeout 8.json -----> DONE
Writeout 9.json -----> DONE
Writeout 10.json -----> DONE
Writeout 11.json -----> DONE
Writeout 12.json -----> DONE
Writeout 13.json -----> DONE
Writeout 14.json -----> DONE
Writeout 15.json -----> DONE
Writeout 16.json -----> DONE
Writeout 17.json -----> DONE
Writeout 18.json -----> DONE
Writeout 19.json -----> DONE
Writeout 20.json -----> DONE
Writeout 21.json -----> DONE
Writeout 22.json -----> DONE
Writeout 23.json -----> DONE
Writeout 24.json -----> DONE
Writeout 25.json -----> DONE
Writeout 26.json -----> DONE
Writeout 27.json -----> DONE
Writeout 28.json -----> DONE
Writeout 29.json -----> DONE
Writeout 30.json -----> DONE
Writeout 31.json -----> DONE
Writeout 32.json -----> DONE
Writeout 33.json -----> DONE
Writeout 34.json -----> DONE
Writeout 35.json ----->

In [11]:
save_to_csv()

Opening json/1.json
======== Saving 1.csv ========
Opening json/10.json
======== Saving 2.csv ========
Opening json/11.json
======== Saving 3.csv ========
Opening json/12.json
======== Saving 4.csv ========
Opening json/13.json
======== Saving 5.csv ========
Opening json/14.json
======== Saving 6.csv ========
Opening json/15.json
======== Saving 7.csv ========
Opening json/16.json
======== Saving 8.csv ========
Opening json/17.json
======== Saving 9.csv ========
Opening json/18.json
======== Saving 10.csv ========
Opening json/19.json
======== Saving 11.csv ========
Opening json/2.json
======== Saving 12.csv ========
Opening json/20.json
======== Saving 13.csv ========
Opening json/21.json
======== Saving 14.csv ========
Opening json/22.json
======== Saving 15.csv ========
Opening json/23.json
======== Saving 16.csv ========
Opening json/24.json
======== Saving 17.csv ========
Opening json/25.json
======== Saving 18.csv ========
Opening json/26.json
======== Saving 19.csv ========
Open